# pydca demo

## MSA Trimming
Here we show how to use pydca as a library. We start with by importing pydca or selected modules within pydca.

In [1]:
"""pydca demo

Author: Mehari B. Zerihun
"""

# import pydca modules
from pydca.plmdca import plmdca
from pydca.meanfield_dca import meanfield_dca
from pydca import sequence_backmapper
from pydca.msa_trimmer import msa_trimmer
from pydca.contact_visualizer import contact_visualizer
from pydca import dca_utilities
import os
import pandas as pd
import numpy as np

print(os.getcwd())

pfam_id = 'PF00186'
pfam_id = 'PF18577'
n_cpu = 26
method = "DI"
preprocess = False # do we want to preprocess FASTA files the same way as in DCA-ER method?

if 0:
    DCA_ER_dir = '/home/eclay/DCA_ER/'
    msa_npy_file = '/home/eclay/Pfam-A.full/%s/msa.npy' % pfam_id # Hurricane Location
    msa_fa_file  = '/home/eclay/Pfam-A.full/%s/msa.fa' % pfam_id # Hurricane Location
    pdb_ref_file = '/home/eclay/Pfam-A.full/%s/pdb_refs.npy' % pfam_id # Hurricane Location
if 1:
    DCA_ER_dir = '/home/evan/PycharmProjects/DCA_ER/'
    msa_npy_file = '/home/evan/PycharmProjects/DCA_ER/Pfam-A.full/%s/msa.npy' % pfam_id
    msa_fa_file  = '/home/evan/PycharmProjects/DCA_ER/Pfam-A.full/%s/msa.fa' % pfam_id
    pdb_ref_file = '/home/evan/PycharmProjects/DCA_ER/Pfam-A.full/%s/pdb_refs.npy' % pfam_id
    out_dir = '%sprotein_data/di/' % DCA_ER_dir
if 0:
    DCA_ER_dir = '/home/ecresswell/DCA_ER/'
    msa_npy_file = '/home/ecresswell/DCA_ER/Pfam-A.full/%s/msa.npy' % pfam_id
    msa_fa_file  = '/home/ecresswell/DCA_ER/Pfam-A.full/%s/msa.fa' % pfam_id
    pdb_ref_file = '/home/ecresswell/DCA_ER/Pfam-A.full/%s/pdb_refs.npy' % pfam_id
    out_dir = '%sprotein_data/di/' % DCA_ER_dir
if 1:
    DCA_ER_dir = '/data/cresswellclayec/DCA_ER/'
    msa_npy_file = '/data/cresswellclayec/DCA_ER/Pfam-A.full/%s/msa.npy' % pfam_id
    msa_fa_file  = '/data/cresswellclayec/DCA_ER/Pfam-A.full/%s/msa.fa' % pfam_id
    pdb_ref_file = '/data/cresswellclayec/DCA_ER/Pfam-A.full/%s/pdb_refs.npy' % pfam_id
    out_dir = '%sprotein_data/di/' % DCA_ER_dir


# os.chdir('/home/evan/PycharmProjects/DCA_ER')
# os.chdir('/home/eclay/DCA_ER') # Hurrican Location
os.chdir(DCA_ER_dir)
ipdb = 0
from ecc_tools import npy2fa
if not preprocess:
    msa_outfile, ref_outfile, s = npy2fa(pfam_id, msa_npy_file, pdb_ref_file=pdb_ref_file, ipdb=ipdb, preprocess=preprocess)
else:
    msa_outfile, ref_outfile, s, cols_removed, s_index, tpdb, orig_seq_len  = npy2fa(pfam_id, msa_npy_file, pdb_ref_file=pdb_ref_file, ipdb=ipdb, preprocess=preprocess,
                                                                                      gap_seqs=.2, gap_cols=.2, prob_low=.004, conserved_cols=.9)

    print('Saved pre-processed msa to %s' % msa_outfile)
    # print(s)
    # print(s.shape)
    gap_pdb = s[tpdb] == '-'  # returns True/False for gaps/no gaps in reference sequence            
    s_gap = s[:, ~gap_pdb]  # removes gaps in reference sequence                                     
    print(gap_pdb)
    ref_s = s_gap[tpdb]
    print(s[tpdb])
    print(s[tpdb].shape)
    print(ref_s)
    print(ref_s.shape)
    print(s.shape)
#     for col_i in range(s.shape[1]):
#         if col_i in cols_removed:
#             print(s[tpdb,col_i])
#             print(s[:,col_i])


print('Preprocessed: %r, s dimensions: ' % preprocess, s.shape)

/gpfs/gsfs8/users/cresswellclayec/DCA_ER
3
After removing bad sequences, tpdb is now  126
Reference sequence (tpdb, s_ipdb) is sequence # 126
shape of s 
import from /data/cresswellclayec/DCA_ER/Pfam-A.full/PF18577/msa.npy
 (129, 52)
/data/cresswellclayec/DCA_ER/Pfam-A.full/PF18577
Preprocessed: False, s dimensions:  (129, 52)


In [2]:
# create MSATrimmer instance 
trimmer = msa_trimmer.MSATrimmer(
    str(msa_outfile), biomolecule='protein', 
    refseq_file=str(ref_outfile),
)

trimmed_data = trimmer.get_msa_trimmed_by_refseq(remove_all_gaps=True)

#write trimmed msa to file in FASTA format
trimmed_data_outfile = '%sprotein_data/data_processing_output/MSA_%s_Trimmed.fa' % (DCA_ER_dir, pfam_id)
with open(trimmed_data_outfile, 'w') as fh:
    for seqid, seq in trimmed_data:
        fh.write('>{}\n{}\n'.format(seqid, seq))
        

In the above, we have imported pydca or selected modules from pydca. Then we use the MSA and reference sequence fasta files (generated from orignal parsed sequences in numpy format). Following that we created an MSATrimmer instance which enabled us to trim the MSA file. Finally, we write the trimmed MSA data to an trimmed output file  in FASTA format. Next we carry out DCA computations using the trimmed MSA.

## DCA Computation Using Pseudolikelihood Maximization Algorithm (plmDCA)

In [3]:
# Compute DCA scores using Pseudolikelihood maximization algorithm

plmdca_inst = plmdca.PlmDCA(
    trimmed_data_outfile,
    'protein',
    seqid = 0.8,
    lambda_h = 1.0,
    lambda_J = 20.0,
    num_threads =n_cpu,
    max_iterations = 500,
)

if preprocess:
    plm_out_file = '%s%s_%s_pydca_plm_preproc.npy' % (out_dir, pfam_id, method)
else:
    plm_out_file = '%s%s_%s_pydca_plm.npy' % (out_dir, pfam_id, method)

if 0: #os.path.exists(plm_out_file):
    plmdca_scores = np.load(plm_out_file)
else:
    # compute PLM DCA scores summarized by various methods:
    # FN_APC: Frobenius norm and average product corrected
    # DI_APC: raw DI with average product correction
    # DI: raw DI score
    # FN: Frobenius norm of raw DI
    if method == "DI_APC":
        plmdca_scores = plmdca_inst.compute_sorted_DI_APC()
    if method == "DI":
        plmdca_scores = plmdca_inst.compute_sorted_DI()
    if method == "FN_APC":
          plmdca_scores = plmdca_inst.compute_sorted_FN_APC()
    if method == "FN":
        plmdca_scores = plmdca_inst.compute_sorted_FN()


In the above, we created a PlmDCA instance `plmdca_inst`  for RNA. Notice that we used the trimmed MSA data we obtained before.  We also set the values of optional parameters. The optional parameters `num_threads` is set to 10. If `pydca` is installed without OpenMP support, we cannot set the number of threads more than one. Finally we computed the DCA scores from the Frobenius norm of the couplings by calling the `compute_sorted_FN_APC()` method on `plmdca_inst`. This action returns the average product corrected (APC) DCA scores. Let us print the top five site pairs and their DCA scores.

In [4]:
# for site_pair, score in plmdca_scores[:5]:
#     print(site_pair, score)
# Save the scores for future use/comparison
np.save(plm_out_file, plmdca_scores)

We have displayed the top five site pairs ranked by their DCA scores. The first column is a tuple of site-pairs and the second column contains the DCA scores obtained from the Frobenius norm of the couplings. Note that the site pairs are arranged in the form (i, j) such that j > i.

## DCA Computation Using Mean-Field Algorithm (mfDCA)

In [5]:
#create mean-field DCA instance 
mfdca_inst = meanfield_dca.MeanFieldDCA(
    trimmed_data_outfile,
    'protein',
    pseudocount = 0.5,
    seqid = 0.8,

)

if preprocess:
    mf_out_file = '%s%s_%s_pydca_mf_preproc.npy' % (out_dir, pfam_id, method)
else:
    mf_out_file = '%s%s_%s_pydca_mf.npy' % (out_dir, pfam_id, method)

if 0: #os.path.exists(mf_out_file):
    mfdca_scores = np.load(mf_out_file)
else:
    # compute MF DCA scores summarized by various methods:
    # FN_APC: Frobenius norm and average product corrected
    # DI_APC: raw DI with average product correction
    # DI: raw DI score
    # FN: Frobenius norm of raw DI
    if method == "DI_APC":
        mfdca_scores = mfdca_inst.compute_sorted_DI_APC()
    if method == "DI":
        mfdca_scores = mfdca_inst.compute_sorted_DI()
    if method == "FN_APC":
        mfdca_scores = mfdca_inst.compute_sorted_FN_APC()
    if method == "FN":
        mfdca_scores = mfdca_inst.compute_sorted_FN()

        


In [6]:
sequences_weight = mfdca_inst.compute_sequences_weight()
fi = mfdca_inst.get_single_site_freqs()
fij = mfdca_inst.get_pair_site_freqs()
reg_fi = mfdca_inst.get_reg_single_site_freqs()                                               
reg_fij = mfdca_inst.get_reg_pair_site_freqs()                                               
corr_mat = mfdca_inst.construct_corr_mat(reg_fi, reg_fij)                                                                             
couplings = mfdca_inst.compute_couplings(corr_mat)        
fields_ij = mfdca_inst.compute_two_site_model_fields(couplings, reg_fi)
np.save('%s%s_pydca_fields_preproc.npy' % (out_dir, pfam_id), fields_ij)
np.save('%s%s_pydca_couplings_preproc.npy' % (out_dir, pfam_id), couplings)
np.save('%s%s_pydca_corr_preproc.npy' % (out_dir, pfam_id), corr_mat)
np.save('%s%s_pydca_unreg_fij_preproc.npy' % (out_dir, pfam_id), fij)
np.save('%s%s_pydca_unreg_fi_preproc.npy' % (out_dir, pfam_id), fi)
np.save('%s%s_pydca_fij_preproc.npy' % (out_dir, pfam_id), reg_fij)
np.save('%s%s_pydca_fi_preproc.npy' % (out_dir, pfam_id), reg_fi)
np.save('%s%s_pydca_seq_weight_preproc.npy' % (out_dir, pfam_id), sequences_weight)
print(couplings.shape)
index_i = []
index_j = []
for [(i, j), score] in mfdca_scores:
    index_i.append(i)
    index_j.append(j)
print(max(index_i))
print(max(index_j))

(940, 940)
45
46


In the above, we created a mean-field DCA instance `mfdca_inst`. We set the optional parameters `pseudocount` and `seqid` to be `0.5` and `0.8`, respectively. Like in the plmDCA we computed the DCA scores from the Frobenius norm of the couplings and average product corrected. using `compute_sorted_FN_APC` method of `mfdca_inst`. Let us print the top five ranked site pairs.

In [7]:
for site_pair, score in mfdca_scores[:5]:
    print(site_pair, score)
    
# Save the scores for future use/comparison
np.save(mf_out_file, mfdca_scores)

(16, 19) 0.473358557698491
(10, 29) 0.4619838846917417
(10, 42) 0.4314340300897064
(17, 31) 0.4234776501104844
(11, 23) 0.40895183531441387


## Plotting Contact Map and True Positive Rates

Next  we plot the contact map and true positive rates of DCA scores computed using the two methods (plmDCA and mfDCA)
### Contact Map and True Positive Rate for PlmDCA Scores

In [8]:
# ---------------------- We need to get pdb information for the contact visualizer --------------------------#
# Define pdb-references for Protein Family

os.chdir(DCA_ER_dir)

# Set DCA_ER directory
DCA_dir = os.getcwd()

pdb = np.load(pdb_ref_file)
processed_data_dir = "%s/protein_data/data_processing_output" % DCA_dir

# delete 'b' in front of letters (python 2 --> python 3)
pdb = np.array([pdb[t,i].decode('UTF-8') for t in range(pdb.shape[0]) \
         for i in range(pdb.shape[1])]).reshape(pdb.shape[0],pdb.shape[1])

# Print number of pdb structures in Protein ID folder
npdb = pdb.shape[0]
print('number of pdb structures:',npdb)

# Create pandas dataframe for protein structure
pdb_df = pd.DataFrame(pdb,columns = ['PF','seq','id','uniprot_start','uniprot_start',\
                                 'pdb_id','chain','pdb_start','pdb_end'])
pdb_df.head()
pdb_id = pdb_df.iloc[ipdb]['pdb_id']
pdb_chain = pdb_df.iloc[ipdb]['chain']
# -----------------------------------------------------------------------------------------------------------#



plmdca_visualizer = contact_visualizer.DCAVisualizer('protein', pdb_chain, pdb_id,
    refseq_file = str(ref_outfile),
    sorted_dca_scores = plmdca_scores,
    linear_dist = 4,
    contact_dist = 5.0,
)

number of pdb structures: 5



	Unable to obtain standared residues for chain E of the PDB file 5J67_downloaded.pdb


PDBContentException: 

Now we have created an instance of `DCAVisualizer`. It takes the type of biomolecule ('rna') , PDB chain ('x'), PDB ID ('1y26'). In addition we have specified reference sequence file, list of sorted DCA scores, linear distance and contact distnace.  When we execute the above cell we see a warning `You didn't supply RNA secondary structure file`. Since supplying an RNA secondary structure file is optional, we can ignore the warning. Next we plot the contact map and true postive rates.

In [ ]:
contact_map_data = plmdca_visualizer.plot_contact_map()

In [ ]:
tp_rate_data = plmdca_visualizer.plot_true_positive_rates()

### Contact Map and True Positive Rate for Mean-Field DCA Scores

Like DCA scores computed using the plmDCA algorithm, we can visualize contact map and true positive rates for mfDCA computed DCA scores as shown below.

In [ ]:
mfdca_visualizer = contact_visualizer.DCAVisualizer('protein', pdb_chain, pdb_id,
    refseq_file = str(ref_outfile),
    sorted_dca_scores = mfdca_scores,
    linear_dist = 4,
    contact_dist = 5.0,
)

In [ ]:
contact_map_data = mfdca_visualizer.plot_contact_map()

In [ ]:
tp_rate_data = mfdca_visualizer.plot_true_positive_rates()

### Computing Fields and Couplings

We can also compute the fields and couplings of the global probability model. That is the $h_{i}$s and $J_{ij}$s in the expression $P(a_{1}, a_{2}, ..., a_{L}) =\frac{1}{Z}\exp(\sum_{i<j} J_{ij}(a_{i}, a_{j}) + \sum_{i} h_{i}(a_{i}))$. In pydca these parameters can be computed using either the mean-field DCA algorithm or the pseudolikelihood maximization DCA algorithm.

In [ ]:
# fields_plm, couplings_plm = plmdca_inst.compute_params(ranked_by='di_apc', num_site_pairs=100)


Or using the mean-field algorithm as:

In [ ]:
# fields_mf, couplings_mf = mfdca_inst.compute_params(ranked_by='fn_apc', num_site_pairs=200)

When we compute the fields and couplings, we can specify a ranking method for the couplings so that the site pairs for the couplings are ordered according to the ranking method.  In the above examples, we used 'di_apc' for the PlmDCA and 'fn_apc' for the mfDCA algorithms. 'di_apc' stands for the ranking method is average product corrected direct information scores, whereas 'fn_apc' implies average corrected Frobenius norm.